<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/07_Evaluacion_Modelos_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluación de Modelos basados en Prompts usando LangChain

# 1) Importación de librerías

In [8]:
!pip install langchain-huggingface
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 2) Configuración del modelo base

Usaremos un modelo base de lenguaje general

In [9]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Modelo pequeño para pruebas
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Creamos un pipeline de generación de texto

In [10]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100,
    truncation=True,
    do_sample=True,
    return_full_text=False,  # Esto hace que solo devuelva el texto nuevo generado
    temperature=0.7
)

# Configurar modelo en LangChain
llm = HuggingFacePipeline(pipeline=text_pipeline)

Device set to use cpu


# 3) Prompt Template

In [11]:
prompt_template = PromptTemplate(
    input_variables=["review"],
    template="""Clasifica el sentimiento de esta reseña como POSITIVO, NEGATIVO o NEUTRAL.
Responde ÚNICAMENTE con una de estas tres palabras.

RESEÑA: {review}

CLASIFICACIÓN:"""
)

# 4) Función de predicción

In [18]:
# Podemos añadir una función de prueba para verificar
def test_prediction(review):
    """Función para probar una predicción individual y ver el proceso"""
    prompt = prompt_template.format(review=review)
    response = llm.invoke(prompt)
    print("###################### Prompt completo:")
    print(prompt)
    print("###################### Respuesta del modelo:")
    print(response)

test_prediction("Este producto superó todas mis expectativas, es increíble!")

###################### Prompt completo:
Clasifica el sentimiento de esta reseña como POSITIVO, NEGATIVO o NEUTRAL.
Responde ÚNICAMENTE con una de las tres palabras y no digas nadas más.

RESEÑA: Este producto superó todas mis expectativas, es increíble!

CLASIFICACIÓN:
###################### Respuesta del modelo:
 POSITIVO | 100%



## 5) Evaluación

In [21]:
# Definimos un conjunto de datos de prueba
test_data = [
    {"review": "Este producto superó todas mis expectativas, es increíble!", "sentiment": "positivo"},
    {"review": "No funciona como esperaba, me decepcionó mucho.", "sentiment": "negativo"},
    {"review": "Es un producto normal, cumple su función básica.", "sentiment": "neutral"},
    {"review": "Excelente calidad y el servicio al cliente es fantástico.", "sentiment": "positivo"},
    {"review": "Terrible experiencia, no lo recomiendo en absoluto.", "sentiment": "negativo"}
]

# Función para evaluar múltiples predicciones
def evaluate_predictions(test_data):
    """Evalúa el modelo con un conjunto de datos de prueba"""
    print("Evaluando predicciones...\n")

    # Para almacenar resultados
    results = []

    for item in test_data:
        # Obtener predicción
        response = llm.invoke(prompt_template.format(review=item["review"]))

        # Almacenar resultados
        results.append({
            "review": item["review"],
            "expected": item["sentiment"],
            "predicted": response.strip().lower(),
            "is_correct": response.strip().lower() == item["sentiment"]
        })

    # Mostrar resultados
    print("Resultados detallados:")
    print("=" * 80)
    for r in results:
        print(f"\nReseña: '{r['review']}'")
        print(f"Esperado: '{r['expected']}'")
        print(f"Predicho: '{r['predicted']}'")
        print(f"Correcto: {'✓' if r['is_correct'] else '✗'}")

    # Calcular accuracy
    accuracy = sum(r['is_correct'] for r in results) / len(results)
    print("\n" + "=" * 80)
    print(f"\nPrecisión total: {accuracy:.2%}")

    return results

# Ejecutar evaluación
results = evaluate_predictions(test_data)

Evaluando predicciones...

Resultados detallados:

Reseña: 'Este producto superó todas mis expectativas, es increíble!'
Esperado: 'positivo'
Predicho: 'positivo (3)

rese'
Correcto: ✗

Reseña: 'No funciona como esperaba, me decepcionó mucho.'
Esperado: 'negativo'
Predicho: 'positivo

reseña: n'
Correcto: ✗

Reseña: 'Es un producto normal, cumple su función básica.'
Esperado: 'neutral'
Predicho: 'positiva

<|user|>
can'
Correcto: ✗

Reseña: 'Excelente calidad y el servicio al cliente es fantástico.'
Esperado: 'positivo'
Predicho: 'positiva, positiva'
Correcto: ✗

Reseña: 'Terrible experiencia, no lo recomiendo en absoluto.'
Esperado: 'negativo'
Predicho: 'positiva
la sensibilidad'
Correcto: ✗


Precisión total: 0.00%


## 8) Ejercicios Propuestos:

1. Experimenta con diferentes prompts y analiza cómo afecta al rendimiento
2. Prueba con otros modelos base de Hugging Face
3. Añade más ejemplos al conjunto de prueba
4. Analiza qué tipos de reseñas son más difíciles para el modelo
5. Compara el rendimiento con diferentes longitudes de reseñas